In [5]:
!pip install ace_tools

ERROR: Could not find a version that satisfies the requirement ace_tools (from versions: none)
ERROR: No matching distribution found for ace_tools


In [6]:
import pandas as pd
import numpy as np
import requests
import json

In [7]:
api_key = 'NYR6U4zsGRBvbJUv4qZEF8h8A8MAmj91'
symbol = 'AAPL'

balance_sheet = '/content/apple_balance_sheet_data_1985_2023.csv'
income_statement = '/content/apple_income_statement_data_1985_2023.csv'
cash_flow = '/content/cash_flow_statement_data_1985_2023.csv'
shares_outstanding = 'https://financialmodelingprep.com/api/v4/shares_float?symbol=A{symbol}&apikey={api_key}'
profile = 'https://financialmodelingprep.com/api/v3/profile/{symbol}?period=annual&apikey={api_key}'

In [8]:
import pandas as pd

# Leer el archivo CSV
file_path_balance = balance_sheet
df_bs = pd.read_csv(file_path_balance)

# Seleccionar las columnas relevantes
df_selected = df_bs[['date', 'totalDebt', 'totalEquity']].copy()

# Convertir la columna date a tipo datetime
df_selected['date'] = pd.to_datetime(df_selected['date'])

# Crear la columna year
df_selected['year'] = df_selected['date'].dt.year

# Calcular las nuevas variables de peso
df_selected['debt_weight'] = df_selected['totalDebt'] / (df_selected['totalDebt'] + df_selected['totalEquity'])
df_selected['equity_weight'] = df_selected['totalEquity'] / (df_selected['totalDebt'] + df_selected['totalEquity'])

# Agrupar por año y calcular los promedios de los pesos
result = df_selected.groupby('year')[['debt_weight', 'equity_weight']].mean().reset_index()

# Convertir a diccionarios
debt_weight_dict = result.set_index('year')['debt_weight'].to_dict()
equity_weight_dict = result.set_index('year')['equity_weight'].to_dict()

# Mostrar los diccionarios
print(debt_weight_dict)
print(equity_weight_dict)


{1985: 0.0, 1986: 0.0, 1987: 0.0, 1988: 0.11305577654026341, 1989: 0.03682333873581848, 1990: 0.07811902637950809, 1991: 0.07758575680050123, 1992: 0.07778574138876007, 1993: 0.2888826501965188, 1994: 0.20022731749582967, 1995: 0.20845839017735335, 1996: 0.35546507986219855, 1997: 0.4485294117647059, 1998: 0.3674884437596302, 1999: 0.0881316098707403, 2000: 0.06807351940095303, 2001: 0.07481708756195421, 2002: 0.07163908410791203, 2003: 0.06715263971725205, 2004: 0.0, 2005: 0.0, 2006: 0.0, 2007: 0.0, 2008: 0.0, 2009: 0.0, 2010: 0.0, 2011: 0.0, 2012: 0.0, 2013: 0.12070401184265776, 2014: 0.2403603873551164, 2015: 0.3506857363573554, 2016: 0.40427162638597924, 2017: 0.4632258426201412, 2018: 0.5165501060325768, 2019: 0.5442214219155312, 2020: 0.6324623822247223, 2021: 0.6640736066961648, 2022: 0.7032230102904399, 2023: 0.6631671373054888}
{1985: 1.0, 1986: 1.0, 1987: 1.0, 1988: 0.8869442234597366, 1989: 0.9631766612641816, 1990: 0.9218809736204919, 1991: 0.9224142431994987, 1992: 0.92221

In [9]:
import pandas as pd

# Definir las rutas de los archivos nuevamente
file_path_balance = balance_sheet
file_path_income = income_statement

# Cargar nuevamente los datos necesarios para asegurar la integridad de las columnas
df_balance = pd.read_csv(file_path_balance)
df_income = pd.read_csv(file_path_income)

# Seleccionar las columnas relevantes para el cálculo del ratio de deuda y el gasto de impuestos
df_income_selected = df_income[['date', 'interestExpense', 'incomeTaxExpense', 'incomeBeforeTax']].copy()
df_balance_selected = df_balance[['date', 'totalDebt']].copy()

# Convertir la columna date a tipo datetime en ambos DataFrames
df_income_selected['date'] = pd.to_datetime(df_income_selected['date'])
df_balance_selected['date'] = pd.to_datetime(df_balance_selected['date'])

# Crear la columna year en ambos DataFrames
df_income_selected['year'] = df_income_selected['date'].dt.year
df_balance_selected['year'] = df_balance_selected['date'].dt.year

# Agrupar por año y calcular la media de interestExpense, incomeTaxExpense, incomeBeforeTax, y totalDebt
df_income_grouped = df_income_selected.groupby('year')[['interestExpense', 'incomeTaxExpense', 'incomeBeforeTax']].mean().reset_index()
df_balance_grouped = df_balance_selected.groupby('year')['totalDebt'].mean().reset_index()

# Fusionar ambos DataFrames en base al año
df_merged = pd.merge(df_income_grouped, df_balance_grouped, on='year')

# Calcular el ratio entre interestExpense y totalDebt
df_merged['debt_cost_ratio'] = df_merged['interestExpense'] / df_merged['totalDebt']

# Calcular el ratio entre incomeTaxExpense e incomeBeforeTax
df_merged['tax_ratio'] = df_merged['incomeTaxExpense'] / df_merged['incomeBeforeTax']

# Reemplazar los valores inf y -inf por 0 y convertir los negativos a positivos
df_merged['debt_cost_ratio'] = df_merged['debt_cost_ratio'].replace([float('inf'), float('-inf')], 0).abs()
df_merged['tax_ratio'] = df_merged['tax_ratio'].replace([float('inf'), float('-inf')], 0).abs()

# Calcular el costo de la deuda después de impuestos
df_merged['after_tax_debt_cost'] = df_merged['debt_cost_ratio'] * (1 - df_merged['tax_ratio'])

# Convertir a diccionario
after_tax_debt_cost_dict = df_merged.set_index('year')['after_tax_debt_cost'].to_dict()

print(after_tax_debt_cost_dict)


{1985: 0.0, 1986: 0.0, 1987: 0.0, 1988: 0.34197784815028315, 1989: 2.362556367359065, 1990: 0.6617664161535391, 1991: 0.43723461533178326, 1992: 0.33402080274455587, 1993: 0.044127948755941684, 1994: 0.04120339800971182, 1995: 0.039523358242577716, 1996: 0.033310087935638596, 1997: 0.053184563495176085, 1998: 0.061038787253158984, 1999: 0.13928500986193296, 2000: 0.05038461538461539, 2001: 0.03591361320067945, 2002: 0.026007565837334497, 2003: 0.019450800915331804, 2004: 0.0, 2005: 0.0, 2006: 0.0, 2007: 0.0, 2008: 0.0, 2009: 0.0, 2010: 0.0, 2011: 0.0, 2012: 0.0, 2013: 0.005921539454107361, 2014: 0.008037283876159925, 2015: 0.00837268600880444, 2016: 0.012453882853772087, 2017: 0.015150009154800227, 2018: 0.02311009796327691, 2019: 0.027819822816742293, 2020: 0.021865584219151, 2021: 0.01838657462514893, 2022: 0.020455298436978075, 2023: 0.027412814168700176}


In [10]:
import pandas as pd
import numpy as np
import yfinance as yf
from pandas_datareader import data as pdr
import datetime

# Reemplazar el DataFrame de pandas_datareader con yfinance
yf.pdr_override()

# Definir el rango de fechas
start_date = datetime.datetime(1985, 1, 1)
end_date = datetime.datetime.today()

# Descargar los datos históricos de AAPL y el S&P 500
aapl = pdr.get_data_yahoo('AAPL', start_date, end_date)
sp500 = pdr.get_data_yahoo('^GSPC', start_date, end_date)
# Descargar los datos históricos de los bonos a 10 años de EE.UU.
us_bond = pdr.get_data_fred('DGS10', start_date, end_date)

# Calcular los retornos diarios
aapl['Returns'] = aapl['Adj Close'].pct_change()
sp500['Returns'] = sp500['Adj Close'].pct_change()

# Crear una nueva columna 'year' para agrupar los datos por año
aapl['year'] = aapl.index.year
sp500['year'] = sp500.index.year
us_bond['year'] = us_bond.index.year

# Calcular el promedio anual de las tasas de los bonos
us_bond['DGS10'] = us_bond['DGS10'] / 100
rf_rate_grouped = us_bond.groupby('year')['DGS10'].mean().reset_index()

# Inicializar un diccionario para almacenar el CAPM
capm_dict = {}

# Calcular la beta y el CAPM por año
for year in range(1985, end_date.year + 1):
    aapl_returns = aapl[aapl['year'] == year]['Returns'].dropna()
    sp500_returns = sp500[sp500['year'] == year]['Returns'].dropna()

    if len(aapl_returns) > 0 and len(sp500_returns) > 0:
        # Calcular la beta
        covariance_matrix = np.cov(aapl_returns, sp500_returns)
        beta = covariance_matrix[0, 1] / covariance_matrix[1, 1]

        # Calcular el rendimiento del mercado
        market_return = sp500[sp500['year'] == year]['Adj Close'].pct_change().mean() * 252

        # Obtener la tasa libre de riesgo
        rf_rate = rf_rate_grouped[rf_rate_grouped['year'] == year]['DGS10'].values[0]

        # Calcular el CAPM
        capm = rf_rate + beta * (market_return - rf_rate)
        capm_dict[year] = capm
    else:
        capm_dict[year] = np.nan

print(capm_dict)


yfinance: pandas_datareader support is deprecated & semi-broken so will be removed in a future verison. Just use yfinance.


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


{1985: 0.2933400952291715, 1986: 0.17396710163624432, 1987: 0.04449631048041282, 1988: 0.09845191455274507, 1989: 0.22617904644665604, 1990: -0.14436510906772554, 1991: 0.2874841869765941, 1992: 0.02924808618655405, 1993: 0.0780726172401041, 1994: -0.02414077598189758, 1995: 0.2642654132491563, 1996: 0.1626479015634981, 1997: 0.23570437512571984, 1998: 0.28670214791440346, 1999: 0.2497236553831979, 2000: -0.17096501617044552, 2001: -0.18518795874754837, 2002: -0.2618433652230282, 2003: 0.24915963945679742, 2004: 0.11490359440143053, 2005: 0.04330957897113771, 2006: 0.16188717521317555, 2007: 0.04951611832038732, 2008: -0.3734706069912828, 2009: 0.19478894061512864, 2010: 0.12567484799070422, 2011: 0.018630967691894366, 2012: 0.14676257512309102, 2013: 0.1596376033482289, 2014: 0.10787915201686285, 2015: 0.0026555185043728516, 2016: 0.11624403534079543, 2017: 0.2270124665244403, 2018: -0.08121234571535448, 2019: 0.3950522938826857, 2020: 0.2259867146591458, 2021: 0.34066117386584327, 20

In [11]:
wacc_dict = {}
for year in debt_weight_dict:
    if year in equity_weight_dict and year in after_tax_debt_cost_dict and year in capm_dict:
        wacc = (debt_weight_dict[year] * after_tax_debt_cost_dict[year]) + (equity_weight_dict[year] * capm_dict[year])
        if wacc < 0:
          wacc = 0.02
        wacc_dict[year] = wacc

# Mostrar el diccionario WACC
print(wacc_dict)

{1985: 0.2933400952291715, 1986: 0.17396710163624432, 1987: 0.04449631048041282, 1988: 0.12598392808330736, 1989: 0.3048475922021341, 1990: 0.02, 1991: 0.29910268729173073, 1992: 0.052955057899084654, 1993: 0.06826659144835702, 1994: 0.02, 1995: 0.21741604625733474, 1996: 0.11667282531306886, 1997: 0.15383887137966715, 1998: 0.2037734706914346, 1999: 0.2399905197614648, 2000: 0.02, 2001: 0.02, 2002: 0.02, 2003: 0.23373408458235395, 2004: 0.11490359440143053, 2005: 0.04330957897113771, 2006: 0.16188717521317555, 2007: 0.04951611832038732, 2008: 0.02, 2009: 0.19478894061512864, 2010: 0.12567484799070422, 2011: 0.018630967691894366, 2012: 0.14676257512309102, 2013: 0.14108345775154613, 2014: 0.08388112191630505, 2015: 0.004660447600542787, 2016: 0.07428462159201771, 2017: 0.12887230118981502, 2018: 0.02, 2019: 0.1951965163056238, 2020: 0.0968877782386634, 2021: 0.12664711840151432, 2022: 0.02, 2023: 0.10384133325981845}
